# Compute diffusion scores in Python

In [1]:
import networkx as nx
import numpy as np
import logging
import scipy as sp
from math import pi, sqrt
import sys
import os 

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

dir_path = os.path.dirname(os.path.realpath('__file__'))

DEBUG:root:test


#### Import kernel functions from diffuPy

The kernel functions a imported from the package. Despite this the functions implementation are in this notebook (final _imp in the function name).

In [2]:
from diffupy.kernel import commute_time_kernel, p_step_kernel, inverse_cosine_kernel, diffusion_kernel, regularised_laplacian_kernel

from diffupy.matrix import Matrix, LaplacianMatrix, csv_to_matrix


### Import example graph

In [3]:
G = nx.read_gml(dir_path+'/04_unit_testing/_graph.gml', label='id')

### General functions

#### Labels mapping

In [4]:
def csv_labeled_matrix_to_matrix(path):
    # Import matrix from csv file and remove headers
    m = np.genfromtxt(path, dtype=None, delimiter=',')
    return Matrix(np.array([[float(x) for x in a[1:]] for a in m[1:]]),os.path.basename(path).split('.csv'), m[1:,0], m[0, 1:])                                                                                              
                                                                                                                    

In [5]:
def run_score_test(score_func, G, input_scores, test_output_scores, z = False):
    
    computed_output_scores = score_func(G, input_scores, z)

    if isinstance(computed_output_scores, Matrix):
        computed_output_scores = computed_output_scores.mat
    
    if isinstance(test_output_scores, Matrix):
        test_output_scores = test_output_scores.mat
    
    logging.info(' %s  \n %s\n', 'Computed matrix', computed_output_scores)
    logging.info(' %s  \n %s\n', 'Test matrix', test_output_scores)
    # Assert rounded similarity (floating comma)
    assert np.allclose(computed_output_scores, test_output_scores)
    logging.info(' Test '+ score_func.__name__ +' passed')

#### Helpers

In [6]:
# In which format is the input? Tell apart vector, matrix or list of matrices
def which_format(x):
    # if is.numeric(x) and is.null(dim(x)): return "vector"
    if isinstance(x, isinstance(x[0], list)) or isinstance(x, isinstance(x[0], np.ndarray)): return "matrix"
    if isinstance(x, list): return "list"
    
    raise ValueError('Non-recognised input scores format, object of class:  %s', x.__class__.__name__)

#### Checks

In [7]:
# TO-DO
#.check_method
#.check_metric

# Labels matrix !!

# def check_scores(scores):
#     form = which_format(scores)
#     #scores_names = names(scores)
#     #if is.null(scores_names):
#     #    raise ValueError("Scores must be a named list, but supplied list contains no names.")

#     plyr::l_ply(
#         scores_names,
#         function(mat_name) {
#             mat <- scores[[mat_name]]

#             if (!is.numeric(mat) & !("dgCMatrix" %in% class(mat))) {
#                 stop(
#                     "The scores in background ",
#                     mat_name,
#                     " are not numeric!"
#                 )
#             }
#             if (any(is.na(mat))) {
#                 stop(
#                     "Scores input cannot contain NA! ",
#                     "But background ",
#                     mat_name,
#                     " does!")
#             }
#             if (is.null(rownames(mat)))
#                 stop(
#                     "The scores in background ",
#                     mat_name,
#                     " must have rownames ",
#                     "according to the scored nodes!"
#                 )
#             if (is.null(colnames(mat)))
#                 stop(
#                     "The scores in background ",
#                     mat_name,
#                     " must have colnames ",
#                     "to differentiate score sets!"
#                 )

#             std <- apply(mat, 2, stats::sd)
#             std_zero <- which(std == 0)
#             std_na <- which(is.na(std))

#             if (length(std_na))
#                 warning(
#                     "Standard deviation in background ",
#                     mat_name,
#                     " is NA in columns: ",
#                     paste(std_na, collapse = ",")
#                 )
#             if (length(std_zero))
#                 warning(
#                     "Standard deviation in background ",
#                     mat_name,
#                     " is 0 in columns: ",
#                     paste(std_zero, collapse = ",")
#                 )
#         }
#     )
#     invisible()
# }

## Diffuse scores

Diffuse scores on a network Function diffuse takes a network in networkx format and an initial state to score all the nodes in the network.

In [8]:
def calculate_scores(col_ind, scores, diff, const_mean, const_var):
    col_in = scores[:, col_ind]
    col_raw = diff[:, col_ind]

    s1 = np.sum(col_in)
    s2 = np.sum(col_in**2)

    # means and vars depend on first and second moments
    # of the input. This should be valid for non-binary
    # inputs as well
    score_means = const_mean*s1
    score_vars = const_var*(len(scores)*s2 - s1**2)
    
    return np.subtract(col_raw, score_means)/np.sqrt(score_vars)

def diffuse_raw (graph,
    scores,
    z = False,
    K = None,
    *argv):
    
    # TODO
    # sanity checks
    # .check_scores(scores)

    # Kernel matrix
    if K is None:
        # .check_graph(graph)
        logging.info('Kernel not supplied. Computing regularised Laplacian kernel ...')
        K = regularised_laplacian_kernel(graph, normalized = False)
        logging.info('Done')
    else:
        # .check_K(K)
        logging.info('Using supplied kernel matrix...')

    # Compute scores        

    # TODO: match indices, for now we assume indices match
    # for i, input_scores in enumerate(scores):
    #    for j, score in enumerate(input_scores):
    #       id_label_mapping(i)
    # input scores

    # TODO: Sparse
    # scores.mat <- methods::as(scores[[scores.name]], "sparseMatrix")

    n = len(scores.mat)
    
    K = K.mat
    
    # raw scores
    diff = np.matmul(K[:,:n], scores.mat)

    # Return base matrix if it is raw
    # Continue if we want z-scores
    if not z: 
        return diff

    # If we want z-scores, must compute rowmeans and rowmeans2
    row_sums = np.array([round(np.sum(row), 2) for row in K[:, :n]])
    row_sums_2 = np.array([np.sum(row) for row in K[:, :n]**2])

    # Constant terms over columns
    const_mean = row_sums/n
    const_var = np.subtract(n*row_sums_2, row_sums**2)/((n - 1)*(n**2))    
    
    return Matrix(np.transpose([np.array(calculate_scores(i, scores.mat, diff, const_mean, const_var)) for i in range(len(diff[0]))]), scores.rows_labels, scores.cols_labels) 

### Raw scores test

In [9]:
run_score_test(diffuse_raw, G, csv_to_matrix(dir_path+'/scores_test/input_scores.csv'), csv_to_matrix(dir_path+'/scores_test/output_scores.csv'))

/Users/jmarinllao/Documents/CREB/diffupy/src/diffupy/matrix.py:117: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  m = np.genfromtxt(path, dtype=None, delimiter=',')
INFO:root:Kernel not supplied. Computing regularised Laplacian kernel ...
INFO:root:Done
INFO:root: Computed matrix  
 [[0.04214556 0.52041526]
 [0.02803012 0.40328263]
 [0.07392846 0.42370717]
 [0.02882746 0.48494791]
 [0.02498394 0.45744575]
 [0.02694918 0.50002046]
 [0.04164558 0.4865847 ]
 [0.02889637 0.52135178]
 [0.03051561 0.42168826]
 [0.02342762 0.49391714]
 [0.02099228 0.45223956]
 [0.02743536 0.4698865 ]
 [0.03170256 0.51293414]
 [0.02845791 0.52145717]
 [0.02020737 0.6197045 ]
 [0.02814164 0.35107263]
 [0.02503623 0.42411564]
 [0.01942714 0.58909861]
 [0.02439709 0.49828292]
 [0.13784437 0.46542244]
 [0.02120247 0.39506635]
 [0.05397049 0.50994295]
 [0.03590104 0.32839363]
 [0.0431597  0.41194104]
 [

### z-scores test

In [10]:
run_score_test(diffuse_raw, G, csv_to_matrix(dir_path+'/scores_test/input_scores.csv'), csv_to_matrix(dir_path+'/scores_test/output_z_scores.csv'), z = True)

INFO:root:Kernel not supplied. Computing regularised Laplacian kernel ...
INFO:root:Done
INFO:root: Computed matrix  
 [[ 5.72900503e-01  9.75391255e-01]
 [-1.13346069e-01 -1.11700155e+00]
 [ 4.08043369e+00 -1.15385656e+00]
 [-9.23887165e-02  6.72815256e-01]
 [-4.09220982e-01 -7.13229608e-02]
 [-2.51410271e-01  1.12880571e+00]
 [ 1.11370470e+00  8.70184615e-01]
 [-5.88803789e-02  1.12033205e+00]
 [ 3.71221934e-02 -9.44090745e-01]
 [-2.57795602e-01  4.55347590e-01]
 [-4.84654538e-01 -1.42913921e-01]
 [-1.25700004e-01  1.65852720e-01]
 [ 1.33054511e-01  1.41590715e+00]
 [-7.37208835e-02  1.00559779e+00]
 [-2.17625149e-01  1.21478023e+00]
 [-6.20089510e-02 -1.24403252e+00]
 [-2.22217600e-01 -5.49847860e-01]
 [-3.76246194e-01  1.57243172e+00]
 [-3.13109721e-01  7.32248463e-01]
 [ 5.05401771e+00  8.69770852e-02]
 [-3.80558984e-01 -9.61394673e-01]
 [ 1.10604763e+00  7.88754532e-01]
 [ 1.31770256e-01 -1.00585729e+00]
 [ 6.24516771e-01 -7.80626191e-01]
 [-3.50165695e-01 -7.15970834e-01]
 [-3.8